In [1]:
!cp -r /kaggle/input/col333-a1/A1/* .

In [2]:
!cat solution.py

import time

class Agent(object):
    def __init__(self, phoneme_table, vocabulary) -> None:
        """
        Your agent initialization goes here. You can also add code but don't remove the existing code.
        """
        self.phoneme_table = phoneme_table
        self.vocabulary = vocabulary
        
        self.inv_phoneme_table = {}
        for phoneme, symbols in phoneme_table.items():
            for symbol in symbols:
                if symbol not in self.inv_phoneme_table:
                    self.inv_phoneme_table[symbol] = []
                self.inv_phoneme_table[symbol].append(phoneme)   
        # need to keep updating this
        self.best_state = None

    def asr_corrector(self, environment):
        """
        Your ASR corrector agent goes here. Environment object has following important members.
        - environment.init_state: Initial state of the environment. This is the text that needs to be corrected.
        - environment.compute_cost: A cost function th

In [3]:
!cat driver.py

import pickle
import json
import argparse
from tqdm import tqdm
from copy import deepcopy

import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor

if torch.cuda.is_available():
    DEVICE = 'cuda'
else:
    DEVICE = 'cpu'


class CostModel(object):
    def __init__(self) -> None:
        # Load Whisper model and processor
        self.__processor = WhisperProcessor.from_pretrained("openai/whisper-small.en")
        self.__model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small.en").to(DEVICE)
        self.__audio_inputs = None

    def set_audio(self, audio, sampling_rate):
        self.__audio_inputs = self.__processor(
            audio, sampling_rate=sampling_rate, return_tensors="pt"
        ).input_features.to(DEVICE)

    def get_loss(self, text):
        # Prepare the target text input IDs
        target = self.__processor(
            text=text, return_tensors="pt", padding=True
        ).input_ids.to(DEVICE)

        # Mak

In [4]:
!cat loss_calc.py

import pickle
import json
import argparse
from tqdm import tqdm
from copy import deepcopy

import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor

if torch.cuda.is_available():
    DEVICE = 'cuda'
else:
    DEVICE = 'cpu'


class CostModel(object):
    def __init__(self) -> None:
        # Load Whisper model and processor
        self.__processor = WhisperProcessor.from_pretrained("openai/whisper-small.en")
        self.__model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small.en").to(DEVICE)
        self.__audio_inputs = None

    def set_audio(self, audio, sampling_rate):
        self.__audio_inputs = self.__processor(
            audio, sampling_rate=sampling_rate, return_tensors="pt"
        ).input_features.to(DEVICE)

    def get_loss(self, text):
        # Prepare the target text input IDs
        target = self.__processor(
            text=text, return_tensors="pt", padding=True
        ).input_ids.to(DEVICE)

        # Mak

In [5]:
!python loss_calc.py --output_file=outputs_super_fin.json

preprocessor_config.json: 100%|██████████████| 185k/185k [00:00<00:00, 2.24MB/s]
tokenizer_config.json: 100%|███████████████████| 805/805 [00:00<00:00, 4.75MB/s]
vocab.json: 100%|████████████████████████████| 798k/798k [00:00<00:00, 2.41MB/s]
tokenizer.json: 100%|██████████████████████| 2.41M/2.41M [00:00<00:00, 5.73MB/s]
merges.txt: 100%|████████████████████████████| 456k/456k [00:00<00:00, 1.85MB/s]
normalizer.json: 100%|██████████████████████| 52.7k/52.7k [00:00<00:00, 662kB/s]
added_tokens.json: 100%|███████████████████| 34.6k/34.6k [00:00<00:00, 73.7MB/s]
special_tokens_map.json: 100%|█████████████| 1.83k/1.83k [00:00<00:00, 10.5MB/s]
config.json: 100%|█████████████████████████| 1.94k/1.94k [00:00<00:00, 11.9MB/s]
model.safetensors: 100%|██████████████████████| 967M/967M [00:02<00:00, 339MB/s]
generation_config.json: 100%|██████████████| 1.93k/1.93k [00:00<00:00, 8.64MB/s]
100%|███████████████████████████████████████████| 71/71 [05:01<00:00,  4.24s/it]
Average error: 1.51579753026

Simple Greedy Approach, No Addition of words (13sec) -> 2.1135895302597905 \
Hil Climbing, No Addition of words (50sec) -> 2.0243456330097898 \
Simple Greedy, Pre - Front and Back Words addition -> 1.88... \
Simple Greedy, Post - Front or Back Words addition -> 1.8454237504744193 \
Simple Greedy, Post - Front and Back Words addition -> 1.8057829830008494 \
Simple Greedy, Post - Front and Back union Front or Back Words addition -> 1.8057829830008494 \
Hill Climbing, Post - Front and Back Words addition ->  1.7099165765332505 \

In [ ]:
!ls

In [ ]:
from IPython.display import FileLink
FileLink(r'outputs.json')